In [1]:
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

data = np.loadtxt('C:\\Git_NEWWWWWWWWW\\StructEst_W19\\ProblemSets\\PS4\\data\\NewMacroSeries.txt', delimiter = ',')
c_t, k_t, w_t, r_t, y_t = data[:,0], data[:,1], data[:,2], data[:,3], data[:,4]


In [2]:
#draw from uniform distribution - 1000 samples S (cols) of 100 data points T (rows)
sim = np.random.uniform(0,1,(100,1000))

In [7]:
#define k_1
k_1 = k_t.mean()

#define simulated variables
def simvar(alpha, beta, rho, mu, sigma, sim):
    eps = sts.norm.ppf(sim,0,sigma)
    z = np.zeros((100,1000))  # of data points (T=100) and # of samples (T=1000)
    z_t_1 = mu
    for i in range(100):   
        z_t = rho*z_t_1 + (1-rho)*mu + eps[i,:]
        z[i,:] = z_t
        z_t_1 = z_t
    
    k = np.zeros((100,1000))
    k_lag = np.zeros((100,1000))
    k_t_1 = k_1
    for i in range(100):
        k_t = alpha*beta*np.exp(z[i,:])*k_t_1**alpha
        k_lag[i,:] = k_t_1
        k[i,:] = k_t
        k_t_1 = k_t

    w = (1-alpha)*np.exp(z)*k_lag**alpha
    r = alpha*np.exp(z)*k_lag**(alpha-1)
    c = w + r*k_lag-k
    y = np.exp(z)*k_lag**alpha
    
    return c,k_lag,k,w,r,y

In [8]:
simvar(0.5,0.99,0.5,8,0.5, sim)

(array([[4342843.5951946 , 4478355.89258913, 7038199.26576965, ...,
         3725170.83290557, 3493803.03778599, 2767514.73022927],
        [3596847.03694698, 4050684.13014104, 2336080.60997248, ...,
         1323453.24956624, 2048956.49892442, 1976838.8255512 ],
        [1700147.72042724, 5925096.32633879, 1828032.16488682, ...,
         1162314.23750792, 2005660.10106935, 2741762.79690983],
        ...,
        [1098138.92222907, 5838501.11148683, 1457177.14119448, ...,
         2376184.53058448,  456764.40136417, 1687277.60349068],
        [ 624244.55471375, 7518705.73374376,  720285.74789716, ...,
         4544247.86773452,  306794.24899828, 3704359.31833287],
        [ 352079.64941627, 8618305.75523742,  234110.15620518, ...,
         3562404.09130721,  614033.63966906, 1696402.03047149]]),
 array([[6643985.13829907, 6643985.13829907, 6643985.13829907, ...,
         6643985.13829907, 6643985.13829907, 6643985.13829907],
        [4256846.69231946, 4389675.57788439, 6898828.98327917

In [50]:
#model moments (mm)

def sim_mom(alpha, beta, rho, mu, sigma, sim):
    c,k_lag,k,w,r,y = simvar(alpha, beta, rho, mu, sigma, sim)
    mm1 = np.mean(c)
    mm2 = np.mean(k)
    mm3 = np.mean(c/y)
    mm4 = np.mean(np.var(y,axis=0))   #changd std into var here
    corr_c = np.zeros((1,1000))
    corr_ck = np.zeros((1,1000))
    
    for i in range(1000):    #need to perform it for the number of samples S, over all T for each sample
        corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        corr_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
    
    mm5 = corr_c.mean()
    mm6 = corr_ck.mean()
    sim_mom = np.array([mm1,mm2,mm3,mm4,mm5,mm6])
    return sim_mom

#test
#print(sim_mom(.5, .99, .5, 10, .5,sim))

#data moments (dm)

dm1 = c_t.mean()
dm2 = k_t.mean()
dm3 = (c_t/y_t).mean()
dm4 = y_t.var()
dm5= np.corrcoef(c_t[0:99],c_t[1:100])[0,1]
dm6= np.corrcoef(c_t, k_t)[0,1]
data_moms=np.array([dm1, dm2, dm3, dm4, dm5, dm6])


def err_vec(alpha, beta, rho, mu, sigma, sim):
    model_moms = sim_mom(alpha, beta, rho, mu, sigma, sim)
    return (model_moms - data_moms)/data_moms

def SMM_crit(params, *args):
    alpha, rho, mu, sigma = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sigma, sim)
    W = np.eye(6)
    return err @ W @ err.T

In [51]:
#SMM simulation
beta=0.99
params_init = np.array([0.5, 0.9, 9, 0.2])
bounds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,1.1))
result = opt.minimize(SMM_crit, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bounds)
alpha_smm, rho_smm, mu_smm, sigma_smm = result.x
print(result)

print('alpha SMM = ', alpha_smm)
print('rho SMM = ', rho_smm)
print('mu SMM = ', mu_smm)
print('sigma SMM = ', sigma_smm)

print('Error vec =', err_vec(alpha_smm, 0.99, rho_smm, mu_smm, sigma_smm, sim))
print("Criterion function value =", SMM_crit(result.x, 0.99, sim))

      fun: 4.314817368462224e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-5.59874522e-05,  6.04842460e-06, -3.54218785e-06,  1.52334199e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 510
      nit: 69
   status: 0
  success: True
        x: array([0.42103658, 0.92218751, 9.93352293, 0.08772051])
alpha SMM =  0.42103658120262505
rho SMM =  0.9221875066859925
mu SMM =  9.933522932906119
sigma SMM =  0.08772050531363076
Error vec = [ 7.35694563e-04 -7.37633645e-04 -1.75661655e-03  2.03365755e-07
  2.70089704e-04 -2.66115161e-04]
Criterion function value = 4.314817368462224e-06


In [52]:
#Jacobian
# number of parameters = 4  (K)
# number of moments = 6  (R)

import numpy.linalg as lin

def Jac_err2(data_vals, unif_vals, alpha, rho, mu, sigma, simple=False):
    '''
    This function computes the Jacobian matrix of partial derivatives of the R x 1 moment
    error vector e(x|theta) with respect to the K parameters theta_i in the K x 1 parameter vector
    theta. The resulting matrix is R x K Jacobian.
    '''
    Jac_err = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_rho = 1e-4 * rho
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    Jac_err[:, 0] = \
        ((err_vec(alpha + h_alpha, 0.99, rho, mu, sigma, sim) -
          err_vec(alpha - h_alpha, 0.99, rho, mu, sigma, sim)) / (2 * h_alpha)).flatten()
    Jac_err[:, 1] = \
        ((err_vec(alpha, 0.99, rho + h_rho, mu, sigma, sim) -
          err_vec(alpha, 0.99, rho - h_rho, mu, sigma, sim)) / (2 * h_rho)).flatten()
    Jac_err[:, 2] = \
        ((err_vec(alpha, 0.99, rho, mu + h_mu, sigma, sim) -
          err_vec(alpha, 0.99, rho, mu - h_mu, sigma, sim)) / (2 * h_mu)).flatten()
    Jac_err[:, 3] = \
        ((err_vec(alpha, 0.99, rho, mu, sigma + h_sigma, sim) -
          err_vec(alpha, 0.99, rho, mu, sigma - h_sigma, sim)) / (2 * h_sigma)).flatten()
    
    return Jac_err

S = sim.shape[1]
d_err2 = Jac_err2(data, sim, alpha_smm, rho_smm, mu_smm, sigma_smm, False)
print(d_err2)
W = np.eye(6)
print(W)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W @ d_err2)
print(SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. rho_hat=', np.sqrt(SigHat2[1, 1]))
print('Std. err. mu_hat=', np.sqrt(SigHat2[2, 2]))
print('Std. err. sigma_hat=', np.sqrt(SigHat2[3, 3]))


[[ 2.70592059e+01  7.71803345e-01  1.71640792e+00  1.46392417e+00]
 [ 3.10890650e+01  7.70667061e-01  1.71388095e+00  1.46176891e+00]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.14461392e+01  1.26421755e+01  3.45978942e+00  2.82786231e+01]
 [ 1.45549368e-01  4.30245058e-01  9.53492765e-04 -8.17683000e-02]
 [ 2.21583477e-01  4.35525999e-01  5.10020952e-03 -7.95879704e-02]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ 8.95641412e-05 -2.70771112e-05 -1.50764712e-03  1.94791858e-06]
 [-2.70771112e-05  2.29151364e-03  2.75423262e-04 -9.99052016e-04]
 [-1.50764712e-03  2.75423262e-04  2.56057046e-02  1.88336206e-05]
 [ 1.94791858e-06 -9.99052016e-04  1.88336206e-05  4.41384501e-04]]
Std. err. alpha_hat= 0.009463833326444146
Std. err. rho_hat= 0.047869757079419505
Std. err. mu_hat= 0.1600178259720861
Std. err. sigma_hat= 0.02100915279823628


In [53]:
# B) TWO STEP WEIGHTING MATRIX

#data moments (dm)

dm1 = c_t.mean()
dm2 = k_t.mean()
dm3 = (c_t/y_t).mean()
dm4 = y_t.var()
dm5= np.corrcoef(c_t[0:99],c_t[1:100])[0,1]
dm6= np.corrcoef(c_t, k_t)[0,1]

def get_Err_mat(unif_vals, alpha, beta, rho, mu, sigma, simple=False):
  
    R = 6
    S = sim.shape[1]
    Err_mat = np.zeros((R, S))
    sim1, sim2, sim3, sim4, sim5, sim6 = sim_mom(alpha, beta, rho, mu, sigma, sim)
    if simple:
        Err_mat[0, :] = sim1 - dm1
        Err_mat[1, :] = sim2 - dm2
        Err_mat[2, :] = sim3 - dm3
        Err_mat[3, :] = sim4 - dm4
        Err_mat[4, :] = sim5 - dm5
        Err_mat[5, :] = sim6 - dm6
    else:
        Err_mat[0, :] = (sim1 - dm1) / dm1
        Err_mat[1, :] = (sim2 - dm2) / dm2
        Err_mat[2, :] = (sim3 - dm3) / dm3
        Err_mat[3, :] = (sim4 - dm4) / dm4
        Err_mat[4, :] = (sim5 - dm5) / dm5
        Err_mat[5, :] = (sim6 - dm6) / dm6
        
    print(sim1, sim2, sim3, sim4, sim5, sim6)
    print(dm1,dm2,dm3,dm4, dm5,dm6)
    return Err_mat


In [40]:
Err_mat = get_Err_mat(sim, alpha_smm, 0.99, rho_smm, mu_smm, sigma_smm, False)
VCV = (1 / sim.shape[1]) * (Err_mat @ Err_mat.T)
print(VCV)
W_hat = lin.inv(VCV)
print(W_hat)

9288581.0885397 6639095.728554834 0.5831723731718768 736697.081376053 0.9408089016009391 0.9405610270361328
9281790.485669706 6643985.138299068 0.5842000000000002 28377825058899.727 0.9405591814596135 0.9408030537975822
[[ 5.35245646e-07 -5.38399134e-07 -1.28691675e-06 -7.31604824e-04
   1.94242392e-07 -1.88209371e-07]
 [-5.38399134e-07  5.41571202e-07  1.29449883e-06  7.35915195e-04
  -1.95386803e-07  1.89318237e-07]
 [-1.28691675e-06  1.29449883e-06  3.09419559e-06  1.75903253e-03
  -4.67026287e-07  4.52520806e-07]
 [-7.31604824e-04  7.35915195e-04  1.75903253e-03  9.99999948e-01
  -2.65501778e-04  2.57255495e-04]
 [ 1.94242392e-07 -1.95386803e-07 -4.67026287e-07 -2.65501778e-04
   7.04911977e-08 -6.83017948e-08]
 [-1.88209371e-07  1.89318237e-07  4.52520806e-07  2.57255495e-04
  -6.83017948e-08  6.61803931e-08]]
[[-3.45308639e+20 -3.73326649e+20  2.64486829e+20 -5.47701372e+17
  -3.79690429e+21 -3.51213933e+21]
 [-3.71339149e+20 -4.24857828e+20  3.21673798e+20 -6.64955819e+17
  -6.3

In [54]:
def SMM_crit2(params, *args):
    alpha, rho, mu, sigma = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sigma, sim)
    return err @ W_hat @ err.T

#SMM simulation
beta=0.99
params_init = np.array([0.5, 0.9, 9, 0.2])
bounds = ((1e-10,1-1e-10),(-1+1e-10,1-1e-10),(5,14),(1e-10,1.1))
result = opt.minimize(SMM_crit2, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bounds)
alpha_smm2, rho_smm2, mu_smm2, sigma_smm2 = result.x
print(result)

print('alpha SMM_2step = ', alpha_smm2)
print('rho SMM_2step = ', rho_smm2)
print('mu SMM_2step = ', mu_smm2)
print('sigma SMM_2step = ', sigma_smm2)


#WHY WON'T THIS CONVERGE....cry


C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in multiply
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in multiply
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in power
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in multiply
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in subtract
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in multiply
C:\Users\avasilj\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in add


      fun: nan
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.65841964e+23, -9.82729299e+22, -3.53061669e+22, -5.54791507e+22])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 105
      nit: 0
   status: 2
  success: False
        x: array([0.5, 0.9, 9. , 0.2])
alpha SMM_2step =  0.5
rho SMM_2step =  0.9
mu SMM_2step =  9.0
sigma SMM_2step =  0.2
